# Labeling ESA tables with VEC labels

In [3]:
import pandas as pd
import numpy as np
import os
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.lf.nlp import nlp_labeling_function
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.apply.dask import DaskLFApplier
from sklearn.model_selection import train_test_split
import spacy
from spacy.matcher import Matcher
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning, ModelPruning
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)

# weights and biases
import wandb

# lightning plus wandb
from pytorch_lightning.loggers import WandbLogger

Global seed set to 42


In [4]:
!nvidia-smi

Thu May 20 03:00:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
esa_df = pd.read_table('/content/drive/My Drive/ESA_tables_figures_5_15_2020_329pm.txt')

In [8]:
len(esa_df)

61247

In [9]:
df = esa_df[esa_df['hasContent'] == 1]
len(df)

52291

In [10]:
df.head()

,csvId,csvFileName,csvFullPath,pdfId,page,tableNumber,topRowJson,titleTag,titleTOC,titleFinal,titleFinal_fr,csvRows,csvColumns,method,accuracy,whitespace,csvText,dt_created,hasContent
105,1059614_14_lattice-v_1,1059614_14_lattice-v_1.csv,\\luxor\data\branch\Environmental Baseline Dat...,1059614,14,1,"[""Survey Date"", ""Aboriginal Communities"", ""Det...",TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table 3 Summary of Aquatics Field Work and Abo...,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,TABLEAU 3 RÉSUMÉ DES SPORTS AQUATIQUES DE TRAV...,4,3,lattice-v,100.0000,0.00000,"[[""Survey Date"", ""Aboriginal Communities"", ""De...",2020-04-07 16:35:41,1
106,1059614_17_lattice-v_1,1059614_17_lattice-v_1.csv,\\luxor\data\branch\Environmental Baseline Dat...,1059614,17,1,"[""Field Site No."", ""Name, Legal Location"", ""...",TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,NaN,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,TABLEAU 4 RÉSUMÉ DES FRANCHISSEMENTS DE COURS ...,10,14,lattice-v,99.8961,9.28571,"[[""Field Site No."", ""Name, Legal Location"", ...",2020-04-07 16:36:05,1
107,1059614_18_lattice-v_1,1059614_18_lattice-v_1.csv,\\luxor\data\branch\Environmental Baseline Dat...,1059614,18,1,"[""Field Site No."", ""Name, Legal Location"", ""...",TABLE 4 Cont'd,NaN,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,TABLEAU 4 RÉSUMÉ DES FRANCHISSEMENTS DE COURS ...,9,14,lattice-v,100.0000,0.00000,"[[""Field Site No."", ""Name, Legal Location"", ...",2020-04-07 16:36:23,1
108,1059614_19_lattice-v_1,1059614_19_lattice-v_1.csv,\\luxor\data\branch\Environmental Baseline Dat...,1059614,19,1,"[""Field Site No."", ""Name, Legal Location"", ""...",TABLE 4 Cont'd,NaN,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,TABLEAU 4 RÉSUMÉ DES FRANCHISSEMENTS DE COURS ...,10,14,lattice-v,100.0000,0.00000,"[[""Field Site No."", ""Name, Legal Location"", ...",2020-04-07 16:36:47,1
109,1059614_20_lattice-v_1,1059614_20_lattice-v_1.csv,\\luxor\data\branch\Environmental Baseline Dat...,1059614,20,1,"[""Field Site No."", ""Name, Legal Location"", ""...",TABLE 4 Cont'd,NaN,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,TABLEAU 4 RÉSUMÉ DES FRANCHISSEMENTS DE COURS ...,8,14,lattice-v,100.0000,0.00000,"[[""Field Site No."", ""Name, Legal Location"", ...",2020-04-07 16:37:02,1
